#### Imports

In [44]:
import os
from pathlib import Path
import copy
from typing import TypeAlias
import numpy as np
import numpy.typing as npt
import py4spice as spi

#### Constants and directories

In [45]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_03/circuits/simple_rc")

# netlist file names
NETLISTS_PATH = PROJ_PATH / "netlists"
TOP_FILENAME = NETLISTS_PATH / "top.cir"
DUT_FILENAME = NETLISTS_PATH / "dut.cir"
LOAD_FILENAME = NETLISTS_PATH / "load.cir"
STIMULUS_FILENAME = NETLISTS_PATH / "stimulus.cir"
SUPPLIES_FILENAME = NETLISTS_PATH / "supplies.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

#### Create netlists objects. We'll combine them after "control" netlist is constructed.

In [46]:
dut = spi.Netlist(NETLISTS_PATH / DUT_FILENAME)
load = spi.Netlist(NETLISTS_PATH / LOAD_FILENAME)
supplies = spi.Netlist(NETLISTS_PATH / SUPPLIES_FILENAME)
stimulus = spi.Netlist(NETLISTS_PATH / STIMULUS_FILENAME)

#### Simulate

In [47]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = dut + load + supplies + stimulus
top.writefile(TOP_FILENAME)
spi.print_section("top netlist", top)

sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run()


--- top netlist ---
R1 IN MID 1K
C1 COM MID 0.1U
R2 MID OUT 100
RLOAD OUT COM 100K
VCOM COM 0 0
VIN IN 0 1
.op
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_03/circuits/simple_rc/netlists/top.cir
-----------------------


Note: No compatibility mode selected!


Circuit: r1 in mid 1k

Doing analysis at TEMP = 27.000000 and TNOM = 27.000000


No. of Data Rows : 1
	Node                                  Voltage
	----                                  -------
	----	-------
	in                               1.000000e+00
	out                              0.000000e+00
	mid                              0.000000e+00
	com                              0.000000e+00

	Source	Current
	------	-------

	vcom#branch                      0.000000e+00
	vin#branch                       0.000000e+00

 Capacitor models (Fixed capacitor)
      model                     C

        cap                     0
         cj                     0
       cjsw                     